<a href="https://colab.research.google.com/github/danielglz096/data-analytics/blob/main/ingenieria_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final Analitica de Datos


*   Jose Daniel Gonzalez - A01751681
*   Victor Eduardo Perez - A01275284

Version 1

##Librerias

In [ ]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from google.colab import drive
from category_encoders import TargetEncoder
import numpy as np
from scipy import stats

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## Preparar Datos para el Modelo

### Demografico

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/analitica/reto_final/demografico.csv')
display(df.head(4))

,id,nacimiento,genero,hijos,ingreso,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,salida,antiguedad,edad_salida,Desc Medida,grupo_persona,evaluacion,funcion,distancia_km,tiempo_h
0,70634620,2/28/1984,Hombre,0,8/24/2022,38,2770,234,Suburbia Los Cabos Patio,10/08/22,0,38,Baja Suburbia,SBB Planta,3.0,Jefe Prevencion Perdidas,16.02733,25.22972222
1,70636679,12/23/1990,Mujer,0,8/31/2022,31,11500,234,Suburbia Los Cabos Patio,11/03/22,0,31,Baja Suburbia,SBB Planta,3.0,Supervisor Cajas,18.52999,27.51166667
2,70642442,6/20/1981,Mujer,0,9/28/2022,41,23410,234,Suburbia Los Cabos Patio,11/22/2022,0,41,Baja Suburbia,SBB Planta,3.0,Auxiliar Cajero,13.77464,22.87666667
3,70639827,05/02/93,Hombre,0,9/14/2022,29,23428,234,Suburbia Los Cabos Patio,10/26/2022,0,29,Baja Suburbia,SBB Planta,3.0,Subjefe Proteccion,0.04798,0.221944444


#### Diseño y construccion

##### Fechas

In [ ]:
df['ingreso'] = pd.to_datetime(df['ingreso'])
df['salida'] = pd.to_datetime(df['salida'])

# Calculate the duration in days and create a new column "duracion"
df['duracion'] = (df['salida'] - df['ingreso']).dt.days
display(df.head(4))

,id,nacimiento,genero,hijos,ingreso,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,salida,antiguedad,edad_salida,Desc Medida,grupo_persona,evaluacion,funcion,distancia_km,tiempo_h,duracion
0,70634620,2/28/1984,Hombre,0,2022-08-24,38,2770,234,Suburbia Los Cabos Patio,2022-10-08,0,38,Baja Suburbia,SBB Planta,3.0,Jefe Prevencion Perdidas,16.02733,25.22972222,45
1,70636679,12/23/1990,Mujer,0,2022-08-31,31,11500,234,Suburbia Los Cabos Patio,2022-11-03,0,31,Baja Suburbia,SBB Planta,3.0,Supervisor Cajas,18.52999,27.51166667,64
2,70642442,6/20/1981,Mujer,0,2022-09-28,41,23410,234,Suburbia Los Cabos Patio,2022-11-22,0,41,Baja Suburbia,SBB Planta,3.0,Auxiliar Cajero,13.77464,22.87666667,55
3,70639827,05/02/93,Hombre,0,2022-09-14,29,23428,234,Suburbia Los Cabos Patio,2022-10-26,0,29,Baja Suburbia,SBB Planta,3.0,Subjefe Proteccion,0.04798,0.221944444,42


In [ ]:
def split_date_column(df, date_column_name, day_column_name, month_column_name, year_column_name):
    df[date_column_name] = pd.to_datetime(df[date_column_name])  # Convert the date column to DateTime type
    df[day_column_name] = df[date_column_name].dt.day
    df[month_column_name] = df[date_column_name].dt.month
    df[year_column_name] = df[date_column_name].dt.year
    df.drop(columns=[date_column_name], inplace=True)  # Drop the original date column

In [ ]:
split_date_column(df,"ingreso","dia_ingreso","mes_ingreso","year_ingreso")
display(df.head(4))

,id,nacimiento,genero,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,salida,antiguedad,...,Desc Medida,grupo_persona,evaluacion,funcion,distancia_km,tiempo_h,duracion,dia_ingreso,mes_ingreso,year_ingreso
0,70634620,2/28/1984,Hombre,0,38,2770,234,Suburbia Los Cabos Patio,2022-10-08,0,...,Baja Suburbia,SBB Planta,3.0,Jefe Prevencion Perdidas,16.02733,25.22972222,45,24,8,2022
1,70636679,12/23/1990,Mujer,0,31,11500,234,Suburbia Los Cabos Patio,2022-11-03,0,...,Baja Suburbia,SBB Planta,3.0,Supervisor Cajas,18.52999,27.51166667,64,31,8,2022
2,70642442,6/20/1981,Mujer,0,41,23410,234,Suburbia Los Cabos Patio,2022-11-22,0,...,Baja Suburbia,SBB Planta,3.0,Auxiliar Cajero,13.77464,22.87666667,55,28,9,2022
3,70639827,05/02/93,Hombre,0,29,23428,234,Suburbia Los Cabos Patio,2022-10-26,0,...,Baja Suburbia,SBB Planta,3.0,Subjefe Proteccion,0.04798,0.221944444,42,14,9,2022


In [ ]:
split_date_column(df,"salida","dia_salida","mes_salida","year_salida")
display(df.head(4))

,id,nacimiento,genero,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,antiguedad,edad_salida,...,funcion,distancia_km,tiempo_h,duracion,dia_ingreso,mes_ingreso,year_ingreso,dia_salida,mes_salida,year_salida
0,70634620,2/28/1984,Hombre,0,38,2770,234,Suburbia Los Cabos Patio,0,38,...,Jefe Prevencion Perdidas,16.02733,25.22972222,45,24,8,2022,8,10,2022
1,70636679,12/23/1990,Mujer,0,31,11500,234,Suburbia Los Cabos Patio,0,31,...,Supervisor Cajas,18.52999,27.51166667,64,31,8,2022,3,11,2022
2,70642442,6/20/1981,Mujer,0,41,23410,234,Suburbia Los Cabos Patio,0,41,...,Auxiliar Cajero,13.77464,22.87666667,55,28,9,2022,22,11,2022
3,70639827,05/02/93,Hombre,0,29,23428,234,Suburbia Los Cabos Patio,0,29,...,Subjefe Proteccion,0.04798,0.221944444,42,14,9,2022,26,10,2022


In [ ]:
split_date_column(df,"nacimiento","dia_nacimiento","mes_nacimiento","year_nacimiento")
display(df.head(4))

,id,genero,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,antiguedad,edad_salida,Desc Medida,...,duracion,dia_ingreso,mes_ingreso,year_ingreso,dia_salida,mes_salida,year_salida,dia_nacimiento,mes_nacimiento,year_nacimiento
0,70634620,Hombre,0,38,2770,234,Suburbia Los Cabos Patio,0,38,Baja Suburbia,...,45,24,8,2022,8,10,2022,28,2,1984
1,70636679,Mujer,0,31,11500,234,Suburbia Los Cabos Patio,0,31,Baja Suburbia,...,64,31,8,2022,3,11,2022,23,12,1990
2,70642442,Mujer,0,41,23410,234,Suburbia Los Cabos Patio,0,41,Baja Suburbia,...,55,28,9,2022,22,11,2022,20,6,1981
3,70639827,Hombre,0,29,23428,234,Suburbia Los Cabos Patio,0,29,Baja Suburbia,...,42,14,9,2022,26,10,2022,2,5,1993


##### One hot encoding

###### Genero

In [ ]:
df = pd.get_dummies(df, columns=['genero'], prefix='', prefix_sep='')
display(df.head(4))

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,antiguedad,edad_salida,Desc Medida,grupo_persona,...,mes_ingreso,year_ingreso,dia_salida,mes_salida,year_salida,dia_nacimiento,mes_nacimiento,year_nacimiento,Hombre,Mujer
0,70634620,0,38,2770,234,Suburbia Los Cabos Patio,0,38,Baja Suburbia,SBB Planta,...,8,2022,8,10,2022,28,2,1984,1,0
1,70636679,0,31,11500,234,Suburbia Los Cabos Patio,0,31,Baja Suburbia,SBB Planta,...,8,2022,3,11,2022,23,12,1990,0,1
2,70642442,0,41,23410,234,Suburbia Los Cabos Patio,0,41,Baja Suburbia,SBB Planta,...,9,2022,22,11,2022,20,6,1981,0,1
3,70639827,0,29,23428,234,Suburbia Los Cabos Patio,0,29,Baja Suburbia,SBB Planta,...,9,2022,26,10,2022,2,5,1993,1,0


###### Tipo de baja

In [ ]:
# Map the values to the desired categories
df['Desc Medida'] = df['Desc Medida'].map({
    'Baja': 'baja_liverpool',
    'Baja por Renuncia Voluntaria': 'baja_liverpool',
    'Baja Suburbia': 'baja_suburbia'
})

# Perform one-hot encoding
df = pd.get_dummies(df, columns=['Desc Medida'], prefix='', prefix_sep='')
display(df.head(4))

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,antiguedad,edad_salida,grupo_persona,evaluacion,...,dia_salida,mes_salida,year_salida,dia_nacimiento,mes_nacimiento,year_nacimiento,Hombre,Mujer,baja_liverpool,baja_suburbia
0,70634620,0,38,2770,234,Suburbia Los Cabos Patio,0,38,SBB Planta,3.0,...,8,10,2022,28,2,1984,1,0,0,1
1,70636679,0,31,11500,234,Suburbia Los Cabos Patio,0,31,SBB Planta,3.0,...,3,11,2022,23,12,1990,0,1,0,1
2,70642442,0,41,23410,234,Suburbia Los Cabos Patio,0,41,SBB Planta,3.0,...,22,11,2022,20,6,1981,0,1,0,1
3,70639827,0,29,23428,234,Suburbia Los Cabos Patio,0,29,SBB Planta,3.0,...,26,10,2022,2,5,1993,1,0,0,1


###### Grupo Persona

In [ ]:
df = pd.get_dummies(df, columns=['grupo_persona'], prefix='', prefix_sep='')
display(df.head(4))

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,antiguedad,edad_salida,evaluacion,funcion,...,baja_liverpool,baja_suburbia,Eventual No Sind.,Eventual Sind.,Planta No Sind.,Planta Sind.,SBB Eventuales,SBB Planta,Vía Planta No Sind.,Vía Planta Sind.
0,70634620,0,38,2770,234,Suburbia Los Cabos Patio,0,38,3.0,Jefe Prevencion Perdidas,...,0,1,0,0,0,0,0,1,0,0
1,70636679,0,31,11500,234,Suburbia Los Cabos Patio,0,31,3.0,Supervisor Cajas,...,0,1,0,0,0,0,0,1,0,0
2,70642442,0,41,23410,234,Suburbia Los Cabos Patio,0,41,3.0,Auxiliar Cajero,...,0,1,0,0,0,0,0,1,0,0
3,70639827,0,29,23428,234,Suburbia Los Cabos Patio,0,29,3.0,Subjefe Proteccion,...,0,1,0,0,0,0,0,1,0,0


##### Rellenar codigos postales

In [ ]:
df['cp_trabajo'] = df['cp_trabajo'].astype(str)
df['cp_trabajo'] = df['cp_trabajo'].str.pad(width=5, side='right', fillchar='0')

df['cp_vivienda'] = df['cp_vivienda'].astype(str)
df['cp_vivienda'] = df['cp_vivienda'].str.pad(width=5, side='right', fillchar='0')

display(df.head(4))

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,ubicacion,antiguedad,edad_salida,evaluacion,funcion,...,baja_liverpool,baja_suburbia,Eventual No Sind.,Eventual Sind.,Planta No Sind.,Planta Sind.,SBB Eventuales,SBB Planta,Vía Planta No Sind.,Vía Planta Sind.
0,70634620,0,38,27700,23400,Suburbia Los Cabos Patio,0,38,3.0,Jefe Prevencion Perdidas,...,0,1,0,0,0,0,0,1,0,0
1,70636679,0,31,11500,23400,Suburbia Los Cabos Patio,0,31,3.0,Supervisor Cajas,...,0,1,0,0,0,0,0,1,0,0
2,70642442,0,41,23410,23400,Suburbia Los Cabos Patio,0,41,3.0,Auxiliar Cajero,...,0,1,0,0,0,0,0,1,0,0
3,70639827,0,29,23428,23400,Suburbia Los Cabos Patio,0,29,3.0,Subjefe Proteccion,...,0,1,0,0,0,0,0,1,0,0


##### Target Encoding

###### Ubicacion

In [ ]:
encoder = TargetEncoder()
encoder.fit(df['ubicacion'], df['antiguedad'])

df['ubicacion_encoded'] = encoder.transform(df['ubicacion'])

df.drop(columns=['ubicacion'],inplace=True)

display(df.head(4))

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,antiguedad,edad_salida,evaluacion,funcion,distancia_km,...,baja_suburbia,Eventual No Sind.,Eventual Sind.,Planta No Sind.,Planta Sind.,SBB Eventuales,SBB Planta,Vía Planta No Sind.,Vía Planta Sind.,ubicacion_encoded
0,70634620,0,38,27700,23400,0,38,3.0,Jefe Prevencion Perdidas,16.02733,...,1,0,0,0,0,0,1,0,0,1.164766
1,70636679,0,31,11500,23400,0,31,3.0,Supervisor Cajas,18.52999,...,1,0,0,0,0,0,1,0,0,1.164766
2,70642442,0,41,23410,23400,0,41,3.0,Auxiliar Cajero,13.77464,...,1,0,0,0,0,0,1,0,0,1.164766
3,70639827,0,29,23428,23400,0,29,3.0,Subjefe Proteccion,0.04798,...,1,0,0,0,0,0,1,0,0,1.164766


###### Funcion

In [ ]:
encoder = TargetEncoder()
encoder.fit(df['funcion'], df['antiguedad'])

df['funcion_encoded'] = encoder.transform(df['funcion'])

df.drop(columns=['funcion'],inplace=True)

display(df.head(4))

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,antiguedad,edad_salida,evaluacion,distancia_km,tiempo_h,...,Eventual No Sind.,Eventual Sind.,Planta No Sind.,Planta Sind.,SBB Eventuales,SBB Planta,Vía Planta No Sind.,Vía Planta Sind.,ubicacion_encoded,funcion_encoded
0,70634620,0,38,27700,23400,0,38,3.0,16.02733,25.22972222,...,0,0,0,0,0,1,0,0,1.164766,2.429671
1,70636679,0,31,11500,23400,0,31,3.0,18.52999,27.51166667,...,0,0,0,0,0,1,0,0,1.164766,2.213115
2,70642442,0,41,23410,23400,0,41,3.0,13.77464,22.87666667,...,0,0,0,0,0,1,0,0,1.164766,0.879807
3,70639827,0,29,23428,23400,0,29,3.0,0.04798,0.221944444,...,0,0,0,0,0,1,0,0,1.164766,2.153846


##### Eliminar personas con duracion menor a 90 dias

In [ ]:
df = df[df['duracion'] >= 90]

##### Limpieza atipicos

##### Guardar archivo

In [ ]:
file_path = '/content/drive/My Drive/analitica/reto_final/demgrafico_transformado.csv'

# Save the DataFrame to the specified file path
df.to_csv(file_path, index=False)

### Historico

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/analitica/reto_final/historico.csv')
display(df2.head(4))

,id,hasta,desde,sociedad,div_personal,grupo_personal,area_personal,ubicacion,unidad_organizacional,funcion,duracion
0,3369,31/12/1998,01/12/1974,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Sin Un Org,Sin Función,8796
1,3369,31/12/1999,01/01/1999,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Compras Juniors,Comprador Sr B,364
2,3369,31/01/2004,01/01/2000,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Compras Juniors,Comprador Sr B,1491
3,3369,30/11/2005,01/02/2004,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Compras Juniors,Comprador Sr A,668


#### Agregar columna de numero de puestos y duracion promedio

In [ ]:
df2['numero_puestos'] = df2.groupby('id')['hasta'].transform('count')
display(df2)

,id,hasta,desde,sociedad,div_personal,grupo_personal,area_personal,ubicacion,unidad_organizacional,funcion,duracion,numero_puestos
0,3369,31/12/1998,01/12/1974,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Sin Un Org,Sin Función,8796,13
1,3369,31/12/1999,01/01/1999,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Compras Juniors,Comprador Sr B,364,13
2,3369,31/01/2004,01/01/2000,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Compras Juniors,Comprador Sr B,1491,13
3,3369,30/11/2005,01/02/2004,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Compras Juniors,Comprador Sr A,668,13
4,3369,31/03/2013,01/12/2005,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,MUJER ACTUAL,Compras Mujer Actual,Comprador Sr A,2677,13
...,...,...,...,...,...,...,...,...,...,...,...,...
442940,70674940,23/12/2022,14/12/2022,Suburbia S. de R.L. de CV,Suburbia Hermosillo Av Solidar,SBB Planta,SBB Operación,JUNIORS SUBURBIA,Jefatura Juniors SBB Herm,Auxiliar Cajero,9,1
442941,70674960,23/12/2022,14/12/2022,Suburbia S. de R.L. de CV,Suburbia Hermosillo Av Solidar,SBB Planta,SBB Operación,CABALLEROS SUBURBIA,Jefatura Caballeros SBB,Auxiliar Cajero,9,1
442942,70675023,23/12/2022,14/12/2022,Suburbia S. de R.L. de CV,Suburbia Hermosillo Av Solidar,SBB Planta,SBB Operación,CAJAS,Jefatura Cajas SBB Hermo,Cajero,9,1
442943,70675038,12/12/2022,12/12/2022,Suburbia S. de R.L. de CV,Suburbia Mahatma Gandhi,SBB Planta,SBB Operación,CABALLEROS SUBURBIA,SUBURBIA MAHATMA GANDHI,Auxiliar Cajero,0,1


In [ ]:
df2['duracion_promedio'] = df2.groupby('id')['duracion'].transform('mean')
display(df2)

,id,hasta,desde,sociedad,div_personal,grupo_personal,area_personal,ubicacion,unidad_organizacional,funcion,duracion,numero_puestos,duracion_promedio
0,3369,31/12/1998,01/12/1974,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Sin Un Org,Sin Función,8796,13,1237.615385
1,3369,31/12/1999,01/01/1999,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Compras Juniors,Comprador Sr B,364,13,1237.615385
2,3369,31/01/2004,01/01/2000,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Compras Juniors,Comprador Sr B,1491,13,1237.615385
3,3369,30/11/2005,01/02/2004,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,JUNIORS,Compras Juniors,Comprador Sr A,668,13,1237.615385
4,3369,31/03/2013,01/12/2005,Servicios Liverpool S.A.,Servicios Liverpool,Planta No Sind.,Ejecutivo/ Coord.,MUJER ACTUAL,Compras Mujer Actual,Comprador Sr A,2677,13,1237.615385
...,...,...,...,...,...,...,...,...,...,...,...,...,...
442940,70674940,23/12/2022,14/12/2022,Suburbia S. de R.L. de CV,Suburbia Hermosillo Av Solidar,SBB Planta,SBB Operación,JUNIORS SUBURBIA,Jefatura Juniors SBB Herm,Auxiliar Cajero,9,1,9.000000
442941,70674960,23/12/2022,14/12/2022,Suburbia S. de R.L. de CV,Suburbia Hermosillo Av Solidar,SBB Planta,SBB Operación,CABALLEROS SUBURBIA,Jefatura Caballeros SBB,Auxiliar Cajero,9,1,9.000000
442942,70675023,23/12/2022,14/12/2022,Suburbia S. de R.L. de CV,Suburbia Hermosillo Av Solidar,SBB Planta,SBB Operación,CAJAS,Jefatura Cajas SBB Hermo,Cajero,9,1,9.000000
442943,70675038,12/12/2022,12/12/2022,Suburbia S. de R.L. de CV,Suburbia Mahatma Gandhi,SBB Planta,SBB Operación,CABALLEROS SUBURBIA,SUBURBIA MAHATMA GANDHI,Auxiliar Cajero,0,1,0.000000


#### Dejar solo un id

In [ ]:
df2_unique = df2.groupby('id')[['numero_puestos', 'duracion_promedio']].first().reset_index()
display(df2_unique)

,id,numero_puestos,duracion_promedio
0,3369,13,1237.615385
1,3798,13,1214.692308
2,4624,8,1916.500000
3,4735,31,521.290323
4,4780,17,959.529412
...,...,...,...
117151,70674940,1,9.000000
117152,70674960,1,9.000000
117153,70675023,1,9.000000
117154,70675038,1,0.000000


### Juntar Informacion (Base para Modelo final)

In [ ]:
result_df = pd.merge(df,df2_unique,on='id',how='left')
display(result_df)

,id,hijos,edad_ingreso,cp_vivienda,cp_trabajo,antiguedad,edad_salida,evaluacion,distancia_km,tiempo_h,...,Planta No Sind.,Planta Sind.,SBB Eventuales,SBB Planta,Vía Planta No Sind.,Vía Planta Sind.,ubicacion_encoded,funcion_encoded,numero_puestos,duracion_promedio
0,13111365,0,21,23450,23400,7,29,3.00,0.33083,0.608888889,...,0,0,0,1,0,0,1.164766,1.329868,5,577.400000
1,70635970,0,41,23473,23400,0,41,3.00,0.4392,0.639166667,...,0,0,0,1,0,0,1.164766,1.406093,1,98.000000
2,70634099,0,26,23477,23400,0,26,3.00,0.34661,0.634444444,...,0,0,0,1,0,0,1.164766,0.782435,1,92.000000
3,70640993,0,24,23477,23400,0,24,3.00,0.34661,0.634444444,...,0,0,0,1,0,0,1.164766,2.540875,1,92.000000
4,13307867,0,18,23562,23400,7,25,3.16,13.77464,22.87666667,...,0,0,0,1,0,0,1.164766,0.799813,15,180.866667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73561,14861927,1,20,Sin Dato,53480,0,21,3.00,0,0,...,0,0,0,0,1,0,1.324040,1.018972,1,89.000000
73562,14864493,0,28,Sin Dato,53480,0,29,3.00,0,0,...,1,0,0,0,0,0,0.678683,1.018972,2,179.500000
73563,15099771,0,21,Sin Dato,51090,0,21,3.00,0,0,...,1,0,0,0,0,0,0.395070,1.018972,2,73.000000
73564,15117343,0,23,Sin Dato,45116,0,24,3.00,5.09188,6.486111111,...,1,0,0,0,0,0,0.722723,1.018972,2,63.000000


#### Revisar tipos de variables

In [ ]:
for column, dtype in result_df.dtypes.iteritems():
    print(f"Column Name: {column}, Data Type: {dtype}")

Column Name: id, Data Type: int64
Column Name: hijos, Data Type: int64
Column Name: edad_ingreso, Data Type: int64
Column Name: antiguedad, Data Type: int64
Column Name: edad_salida, Data Type: int64
Column Name: evaluacion, Data Type: float64
Column Name: distancia_km, Data Type: object
Column Name: tiempo_h, Data Type: object
Column Name: duracion, Data Type: int64
Column Name: dia_ingreso, Data Type: int64
Column Name: mes_ingreso, Data Type: int64
Column Name: year_ingreso, Data Type: int64
Column Name: dia_salida, Data Type: int64
Column Name: mes_salida, Data Type: int64
Column Name: year_salida, Data Type: int64
Column Name: dia_nacimiento, Data Type: int64
Column Name: mes_nacimiento, Data Type: int64
Column Name: year_nacimiento, Data Type: int64
Column Name: Hombre, Data Type: uint8
Column Name: Mujer, Data Type: uint8
Column Name: baja_liverpool, Data Type: uint8
Column Name: baja_suburbia, Data Type: uint8
Column Name: Eventual No Sind., Data Type: uint8
Column Name: Eventu

<ipython-input-57-6ff5e6df466a>:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, dtype in result_df.dtypes.iteritems():


In [ ]:
columns_to_drop = ['cp_vivienda', 'cp_trabajo']  # List of column names to drop
result_df = result_df.drop(columns=columns_to_drop)

#### Guardar archivo final

In [ ]:
file_path = '/content/drive/My Drive/analitica/reto_final/base_final.csv'

# Save the DataFrame to the specified file path
result_df.to_csv(file_path, index=False)